In [3]:
#install libraries
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import tueplots

import sys
print(sys.executable)

/Users/madelinemiller/Desktop/data_literacy/geonews_femicide/source/.venv/bin/python
